Demo to detect Reading in sample video

In [1]:
import os
from moviepy.editor import VideoFileClip, CompositeVideoClip
import json
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

import keras
from keras.models import load_model
import numpy as np
from sklearn.metrics import accuracy_score

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3629056/45929032 bytes (7.9%)7520256/45929032 bytes (16.4%)11689984/45929032 bytes (25.5%)15523840/45929032 bytes (33.8%)19431424/45929032 bytes (42.3%)23486464/45929032 bytes (51.1%)27492352/45929032 bytes (59.9%)31629312/45929032 bytes (68.9%)35536896/45929032 bytes (77.4%)39436288/45929032 bytes (85.9%)43393024/45929032 bytes (94.5%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive/')
# drive.flush_and_unmount()

Mounted at /content/drive/


In [59]:
# All helper functions

def ExtractFrames(file_path, pos=[0,0.05,0.1,0.15,.02,.25,.30,.35,.40,.45,.50,.55,.60,.65,.70,.75,.80,.85,.9,.95]):
    # Extracts frames from file_path at the positions (relative between 0 and 1) in pos
    
    import os
    
    if not len(pos):
        print("[ExtractFrames]: Invalid positions")
        return None
    
    if not os.path.isfile(file_path) :
        print("[ExtractFrames]: Invalid file path")
        return None
    
    import cv2
    
    # container for frames
    arr = np.empty((len(pos),224,224,3))
    
    cap = cv2.VideoCapture(file_path)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    for k,i in enumerate(pos):
        # get frame number
        position = int(i * total_frames)
        
        # set frame pointer at i and extract frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        
        # preprocessing
        frame = cv2.resize(frame, (224,224))
        frame = frame * 1/255.
        frame = np.float32(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # insert in container
        arr[k] = frame
        
    # cleanup
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    cap.release()
    
    return arr

def addJSONEntry(vid, a, b, fname, action, confidence):
    # Adds new entry for 2 second clip to valid file
    
    if not os.path.isfile(fname) :
        print("[addJSONEntry]: Invalid file path")
        return
    
    # hardcoded
    entry = {"videoId": vid, "type": "segment", "startTime": a, "endTime": b, "observer": "CSCE636Spring2021-nini16-demo", "isHuman": True, "observation": {"label": action, "labelConfidence": str(confidence)}}
    with open(fname, 'r') as f:
        data = json.load(f)
    data.append(entry)
    with open(fname, 'w') as f:
        json.dump(data, f, indent=2)

def predict(model, videopath):
    # pass video to model for prediction
    region_clip = ExtractFrames(videopath)
    region_clip = np.expand_dims(region_clip, axis=0)
    pred = model(region_clip).numpy()

    index_max = np.argmax(pred, axis=1)

    # the action we are trying to detect is represented by index 0
    # Hence. we take the inverse of index_max to get a 1 for the detected action
    return (int(not index_max), pred[0][index_max][0])


def Make_Region(path, vid, output, model, action):
    # extract a 2 second clip from video starting from each time in times
    if output == "" or output==None:
        print("[Make_Region]: Please specify a file path")
        return
    
    full_vid = VideoFileClip(path)
    vid_end_time = math.floor(full_vid.duration)
    times = list(range(0, vid_end_time, 2))

    for time in times:
        a = time
        if time == times[-1]:
            if vid_end_time < (a + 2):
                break
            else:
                b = vid_end_time
        else:
            b = a+2
        clip = VideoFileClip(path).subclip(a,b) # mp4 or mkv
        video = CompositeVideoClip([clip])
        # Write the result to a file (many options available !)
        pathto = os.path.join(output, "{}_time_{}-{}.mp4".format(vid, a, b))
        video.write_videofile(pathto)
        clip.close()

        result = predict(model, pathto)
        print("###########################################################")
        print("prediction: {}".format(result))
        print("###########################################################")
        if result[0]:
            addJSONEntry(vid, a, b, "ResultOfDetection.json", "Reading", result[1])

In [72]:
# Load our already trained model
model = load_model(r"/content/drive/MyDrive/CSCE636/v6/new_model_V6_lr_0.0005.h5")

In [70]:
# delete directory and json file if it exists
!rm -rf clips
!rm ResultOfDetection.json

In [73]:
# create JSON file and directory to hold clips
!mkdir clips
!touch ResultOfDetection.json
!echo "[]" > ResultOfDetection.json

In [74]:
Make_Region("/content/drive/MyDrive/CSCE636/v10/BYlSjcgZl7Y.mp4", "BYlSjcgZl7Y", "/content/clips", model, "Reading")

[MoviePy] >>>> Building video /content/clips/BYlSjcgZl7Y_time_0-2.mp4
[MoviePy] Writing audio in BYlSjcgZl7Y_time_0-2TEMP_MPY_wvf_snd.mp3




100%|██████████| 45/45 [00:00<00:00, 1037.96it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/clips/BYlSjcgZl7Y_time_0-2.mp4





  0%|          | 0/60 [00:00<?, ?it/s]

 28%|██▊       | 17/60 [00:00<00:00, 167.15it/s]

 55%|█████▌    | 33/60 [00:00<00:00, 163.38it/s]

 80%|████████  | 48/60 [00:00<00:00, 156.56it/s]

100%|██████████| 60/60 [00:00<00:00, 121.09it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/clips/BYlSjcgZl7Y_time_0-2.mp4 

###########################################################
prediction: (1, 0.9993)
###########################################################
[MoviePy] >>>> Building video /content/clips/BYlSjcgZl7Y_time_2-4.mp4
[MoviePy] Writing audio in BYlSjcgZl7Y_time_2-4TEMP_MPY_wvf_snd.mp3


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/clips/BYlSjcgZl7Y_time_220-222.mp4 

###########################################################
prediction: (1, 0.99946576)
###########################################################
[MoviePy] >>>> Building video /content/clips/BYlSjcgZl7Y_time_222-224.mp4
[MoviePy] Writing audio in BYlSjcgZl7Y_time_222-224TEMP_MPY_wvf_snd.mp3




  0%|          | 0/45 [00:00<?, ?it/s]

100%|██████████| 45/45 [00:00<00:00, 241.71it/s]


[MoviePy] Done.
[MoviePy] Writing video /content/clips/BYlSjcgZl7Y_time_222-224.mp4




  0%|          | 0/60 [00:00<?, ?it/s]

 27%|██▋       | 16/60 [00:00<00:00, 159.65it/s]

 53%|█████▎    | 32/60 [00:00<00:00, 157.66it/s]

 78%|███████▊  | 47/60 [00:00<00:00, 153.97it/s]

100%|██████████| 60/60 [00:00<00:00, 119.89it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/clips/BYlSjcgZl7Y_time_222-224.mp4 

###########################################################
prediction: (1, 0.9987583)
###########################################################
